**The following solution is a content based filtering recommendation system**

It recommends books to a user based on the kinds of books they have read in the past.

In [25]:
import numpy as np
import pandas as pd
import nltk
import json
import re
import csv
from tqdm import tqdm
pd.set_option('display.max_colwidth', 300)

data = []

with open("./data/booksummaries.txt", 'r', encoding='utf8') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)


16559it [00:01, 11403.33it/s]


In [26]:
book_index = []
book_id = []
book_author = []
book_name = []
summary = []
genre = []
a = 1

for i in tqdm(data):
    book_index.append(a)
    a = a+1
    book_id.append(i[0])
    book_name.append(i[2])
    book_author.append(i[3])
    genre.append(i[5])
    summary.append(i[6])

    df = pd.DataFrame({'Index': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                       'Genre': genre, 'Summary': summary})
df.head()


100%|██████████| 16559/16559 [04:48<00:00, 57.31it/s]


,Index,ID,BookTitle,Author,Genre,Summary
0,1,620,Animal Farm,George Orwell,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"": ""Satire"", ""/m/0dwly"": ""Children's literature"", ""/m/014dfn"": ""Speculative fiction"", ""/m/02xlf"": ""Fiction""}","Old Major, the old boar on the Manor Farm, calls the animals on the farm for a meeting, where he compares the humans to parasites and teaches the animals a revolutionary song, 'Beasts of England'. When Major dies, two young pigs, Snowball and Napoleon, assume command and turn his dream into a p..."
1,2,843,A Clockwork Orange,Anthony Burgess,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""Novella"", ""/m/014dfn"": ""Speculative fiction"", ""/m/0c082"": ""Utopian and dystopian fiction"", ""/m/06nbt"": ""Satire"", ""/m/02xlf"": ""Fiction""}","Alex, a teenager living in near-future England, leads his gang on nightly orgies of opportunistic, random ""ultra-violence."" Alex's friends (""droogs"" in the novel's Anglo-Russian slang, Nadsat) are: Dim, a slow-witted bruiser who is the gang's muscle; Georgie, an ambitious second-in-command; and..."
2,3,986,The Plague,Albert Camus,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fiction"", ""/m/0pym5"": ""Absurdist fiction"", ""/m/05hgj"": ""Novel""}","The text of The Plague is divided into five parts. In the town of Oran, thousands of rats, initially unnoticed by the populace, begin to die in the streets. A hysteria develops soon afterward, causing the local newspapers to report the incident. Authorities responding to public pressure order t..."
3,4,1756,An Enquiry Concerning Human Understanding,David Hume,,"The argument of the Enquiry proceeds by a series of incremental steps, separated into chapters which logically succeed one another. After expounding his epistemology, Hume explains how to apply his principles to specific topics. In the first section of the Enquiry, Hume provides a rough introdu..."
4,5,2080,A Fire Upon the Deep,Vernor Vinge,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90"": ""Science Fiction"", ""/m/014dfn"": ""Speculative fiction"", ""/m/01hmnh"": ""Fantasy"", ""/m/02xlf"": ""Fiction""}","The novel posits that space around the Milky Way is divided into concentric layers called Zones, each being constrained by different laws of physics and each allowing for different degrees of biological and technological advancement. The innermost, the ""Unthinking Depths"", surrounds the galacti..."


In [49]:
# Remove duplicates from the dataset
books = df.drop_duplicates(subset='BookTitle')
# df[df.duplicated('BookTitle')].sort_values('BookTitle')

books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16277 entries, 0 to 16558
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Index      16277 non-null  int64 
 1   ID         16277 non-null  object
 2   BookTitle  16277 non-null  object
 3   Author     16277 non-null  object
 4   Genre      16277 non-null  object
 5   Summary    16277 non-null  object
dtypes: int64(1), object(5)
memory usage: 890.1+ KB


In [54]:
books.isna().sum()

books = books.drop(books[books['Genre'] == ''].index)
books = books.drop(books[books['Summary'] == ''].index)


def clean_genre(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = ' '.join(text.split())
    text = text.lower()
    return text


genres_cleaned = []
for i in books['Genre']:
    genres_cleaned.append(list(json.loads(i).values()))
books['Genre'] = genres_cleaned
books.head(2)


,Index,ID,BookTitle,Author,Genre,Summary,Genres,clean_summary
10438,10439,12633939,Jackie & Me,Dan Gutman,"[Children's literature, Fantasy, Fiction, Religion]","Joe Stosh is severely insulted during one of his at-bats in a baseball game by the opposing pitcher, Bobby Fuller, and loses his temper. He charges the mound, determined to teach Fuller a lesson. Mass chaos follows, and Joe ends up getting suspended indefinitely from Little League. He then goes...","[Children's literature, Fantasy, Fiction, Religion]",joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...
7587,7588,6864185,Conan the Wanderer,L. Sprague de Camp,"[Fantasy, Speculative fiction, Short story, Sword and sorcery]","Conan, now about thirty-one, survives a Turanian trap that crushes his Zuagir raiders and seeks bloody revenge on Vardanes of Zamora, their betrayer. Afterwards he moves on to other adventures, engaging in skull-duggery in the cannibal-haunted city of Zamboula and ultimately gaining command of ...","[Fantasy, Speculative fiction, Short story, Sword and sorcery]",conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...


In [57]:
def clean_summary(text):
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]", " ", text)
    text = ' '.join(text.split())
    text = text.lower()
    return text


books['clean_summary'] = books['Summary'].apply(lambda x: clean_summary(x))
books.head(2)


,Index,ID,BookTitle,Author,Genre,Summary,Genres,clean_summary
10438,10439,12633939,Jackie & Me,Dan Gutman,"[Children's literature, Fantasy, Fiction, Religion]","Joe Stosh is severely insulted during one of his at-bats in a baseball game by the opposing pitcher, Bobby Fuller, and loses his temper. He charges the mound, determined to teach Fuller a lesson. Mass chaos follows, and Joe ends up getting suspended indefinitely from Little League. He then goes...","[Children's literature, Fantasy, Fiction, Religion]",joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...
7587,7588,6864185,Conan the Wanderer,L. Sprague de Camp,"[Fantasy, Speculative fiction, Short story, Sword and sorcery]","Conan, now about thirty-one, survives a Turanian trap that crushes his Zuagir raiders and seeks bloody revenge on Vardanes of Zamora, their betrayer. Afterwards he moves on to other adventures, engaging in skull-duggery in the cannibal-haunted city of Zamboula and ultimately gaining command of ...","[Fantasy, Speculative fiction, Short story, Sword and sorcery]",conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...


In [88]:
# books2 = books.loc[:, ['BookTitle', 'Genre', 'clean_summary', 'Author']]

books2 =books2.rename(columns={'BookTitle':'Title', 'clean_summary': 'Summary', 'index': 'Index'})

books2

,level_0,Index,Title,Genre,Summary,Author,Genre_String,combined_summary,combined_genre,combined_two,combined_text
0,0,0,Jackie & Me,"[Children's literature, Fantasy, Fiction, Religion]",joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...,Dan Gutman,Children's literature Fantasy Fiction Religion,joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...,Children's literature Fantasy Fiction Religion,joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...,joe stosh is severely insulted during one of his at bats in a baseball game by the opposing pitcher bobby fuller and loses his temper he charges the mound determined to teach fuller a lesson mass chaos follows and joe ends up getting suspended indefinitely from little league he then goes to scho...
1,1,1,Conan the Wanderer,"[Fantasy, Speculative fiction, Short story, Sword and sorcery]",conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...,L. Sprague de Camp,Fantasy Speculative fiction Short story Sword and sorcery,conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...,Fantasy Speculative fiction Short story Sword and sorcery,conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...,conan now about thirty one survives a turanian trap that crushes his zuagir raiders and seeks bloody revenge on vardanes of zamora their betrayer afterwards he moves on to other adventures engaging in skull duggery in the cannibal haunted city of zamboula and ultimately gaining command of a band...
2,2,2,Beautiful Losers,[Fiction],at the centre of the novel are the members of a love triangle united by their obsessions and fascination with a th century mohawk saint kateri tekakwitha the triangle is made up of the unnamed narrator an authority on the vanishing a tribe his wife edith one of the last surviving members of the ...,Leonard Cohen,Fiction,at the centre of the novel are the members of a love triangle united by their obsessions and fascination with a th century mohawk saint kateri tekakwitha the triangle is made up of the unnamed narrator an authority on the vanishing a tribe his wife edith one of the last surviving members of the ...,Fiction,at the centre of the novel are the members of a love triangle united by their obsessions and fascination with a th century mohawk saint kateri tekakwitha the triangle is made up of the unnamed narrator an authority on the vanishing a tribe his wife edith one of the last surviving members of the ...,at the centre of the novel are the members of a love triangle united by their obsessions and fascination with a th centur

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

books2['Genre_String'] = books2['Genre'].apply(lambda x: ' '.join(x))

# summary
books2["combined_summary"] = books2["Summary"]

# genre
books2["combined_genre"] = books2["Genre_String"]

# genre + summary
books2["combined_two"] = books2["Summary"] + " " + books2["Genre_String"]

# summary + author + genre
books2["combined_text"] = books2["Summary"] + " " + books2["Author"] + " " + books2["Genre_String"]

tf = TfidfVectorizer(analyzer="word", ngram_range=(1, 2),
                     min_df=1, stop_words='english')
tf_char = TfidfVectorizer(analyzer="ch", ngram_range=(
    1, 2), min_df=1, stop_words='english')

tfidf_matrix = tf.fit_transform(books2['combined_text'])
tfidf_matrix_summary = tf.fit_transform(books2['combined_summary'])
tfidf_matrix_genre = tf.fit_transform(books2['combined_genre'])
tfidf_matrix_two = tf.fit_transform(books2['combined_two'])

cosine = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_summary = cosine_similarity(tfidf_matrix_summary, tfidf_matrix_summary)
cosine_genre = cosine_similarity(tfidf_matrix_genre, tfidf_matrix_genre)
cosine_two = cosine_similarity(tfidf_matrix_two, tfidf_matrix_two)


In [89]:
# index로 제목 찾기
def get_title_from_index(Index):
    return books2[books2.Index == Index]["Title"].values[0]
# 제목에서 index 찾기


def get_index_from_title(Title):
    return books2[books2.Title == Title]["Index"].values[0]

# 추천 받기


def get_recommendations(book):
    book_index = get_index_from_title(book)

    # change between cosine, cosine_summary, cosine_genre, cosine_two
    similar_books = list(enumerate(cosine[book_index]))
    sortedbooks = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:]
    i = 0
    for book in sortedbooks:
        print(get_title_from_index(book[0]) +
              " by " + books2.Author[books2["Index"] == book[0]])
        i = i+1
        if i > 10:
            break


def get_recommendations_summary(book):
    book_index = get_index_from_title(book)

    # change between cosine, cosine_summary, cosine_genre, cosine_two
    similar_books = list(enumerate(cosine_summary[book_index]))
    sortedbooks = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:]
    i = 0
    for book in sortedbooks:
        print(get_title_from_index(book[0]) +
              " by " + books2.Author[books2["Index"] == book[0]])
        i = i+1
        if i > 10:
            break


def get_recommendations_genre(book):
    book_index = get_index_from_title(book)

    # change between cosine, cosine_summary, cosine_genre, cosine_two
    similar_books = list(enumerate(cosine_genre[book_index]))
    sortedbooks = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:]
    i = 0
    for book in sortedbooks:
        print(get_title_from_index(book[0]) +
              " by " + books2.Author[books2["Index"] == book[0]])
        i = i+1
        if i > 10:
            break


def get_recommendations_two(book):
    book_index = get_index_from_title(book)

    # change between cosine, cosine_summary, cosine_genre, cosine_two
    similar_books = list(enumerate(cosine_two[book_index]))
    sortedbooks = sorted(similar_books, key=lambda x: x[1], reverse=True)[1:]
    i = 0
    for book in sortedbooks:
        print(get_title_from_index(book[0]) +
              " by " + books2.Author[books2["Index"] == book[0]])
        i = i+1
        if i > 10:
            break


In [95]:
print(get_recommendations("Animal Farm"))


4086    Freddy Goes to Florida by Walter R. Brooks
Name: Author, dtype: object
488    The Animals of Farthing Wood by Colin Dann
Name: Author, dtype: object
4444    The True Story of the Three Little Pigs by Lane Smith
Name: Author, dtype: object
6588    Fire and Sword by Simon Scarrow
Name: Author, dtype: object
873    Freddy the Politician by Walter R. Brooks
Name: Author, dtype: object
2784    Animal World by Antonio di Benedetto
Name: Author, dtype: object
6631    Zoobreak by Gordon Korman
Name: Author, dtype: object
6830    Freddy Rides Again by 
Name: Author, dtype: object
4934    The Siege of White Deer Park by Colin Dann
Name: Author, dtype: object
949    Punk Farm by Jarrett Krosoczka
Name: Author, dtype: object
1982    Guns, Germs, and Steel by Jared Diamond
Name: Author, dtype: object
None


In [141]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorized = vectorizer.fit_transform(df3['data'])


In [142]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity(vectorized)


print(similarities)

[[1.         0.         0.         ... 0.         0.12309149 0.0836242 ]
 [0.         1.         0.         ... 0.         0.         0.18490007]
 [0.         0.         1.         ... 0.14285714 0.         0.        ]
 ...
 [0.         0.         0.14285714 ... 1.         0.         0.        ]
 [0.12309149 0.         0.         ... 0.         1.         0.        ]
 [0.0836242  0.18490007 0.         ... 0.         0.         1.        ]]


In [145]:
df = pd.DataFrame(similarities, columns=df['Book-Title'], index=df['Book-Title']).reset_index()

df.head()


Book-Title,Book-Title,stolen figs : and other adventures in calabria,mungo goes east: a window board book,people of the earth,the burdens of sister margaret,the renegades: rafe,devil's race,love's wild desire,downhill chance,clod hans,...,maison ikkoku : family affairs (maison ikkoku),angels flight (detective harry bosch mysteries),tief wie der ozean.,todo un caracter,heroes' square budapest: hungary's history in stone and bronze,"niv holy bible, textbook edition",gross anatomy: an off-color coloring book,the messies manual,riotous assembly,little house in brookfield (little house)
0,stolen figs : and other adventures in calabria,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.174078,0.0,0.000000,0.000000,0.123091,0.083624
1,mungo goes east: a window board book,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.096225,0.000000,0.000000,0.184900
2,people of the earth,0.0,0.0,1.000000,0.239046,0.308607,0.169031,0.154303,0.154303,0.169031,...,0.0,0.125988,0.0,0.0,0.000000,0.0,0.109109,0.142857,0.000000,0.000000
3,the burdens of sister margaret,0.0,0.0,0.239046,1.000000,0.129099,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.358569,0.000000,0.000000
4,the renegades: rafe,0.0,0.0,0.308607,0.129099,1.000000,0.365148,0.166667,0.166667,0.182574,...,0.0,0.136083,0.0,0.0,0.000000,0.0,0.117851,0.154303,0.000000,0.000000


In [155]:
input_book = 'far beyond the stars (star trek deep space nine)'
recommendations = pd.DataFrame(df.nlargest(11,input_book)['Book-Title'])
recommendations = recommendations[recommendations['Book-Title']!=input_book]
print(recommendations)

                                              Book-Title
1293   the way of the warrior (star trek deep space n...
8930                 quotable star trek (star trek: all)
11406             prime directive: star trek (star trek)
1383   the star trek chronology : a history of the fu...
576    the fall of terok nor (star trek deep space ni...
3565   compendium: star trek r (star trek next genera...
1051   dreams of the raven: star trek #34 (star trek ...
8963            reunion (star trek: the next generation)
9338                                  gemini (star trek)
10202               the joy machine (star trek, book 80)
